# Import Customized modules and Create ImageJ Object

In [ ]:
import os
import sys
import re
from pathlib import Path
from typing import List, Dict, Union, TextIO
from datetime import datetime
import toml
import tomlkit

import logging
from skimage.io import imread, imshow, imsave

abs_module_path = Path("./../../").resolve()
if (abs_module_path.exists()) and (str(abs_module_path) not in sys.path):
    sys.path.append(str(abs_module_path)) # add path to scan customized module

from modules.misc.logger import init_logger
from modules.misc.utils import create_new_dir, get_repo_root, load_config
from modules.data.lif import LIFNameChecker, scan_lifs_under_dir
from modules.data.composite_ij import dump_info, median_R1_and_mean3D_R2, save_tif_with_SN, \
                                      group_show, group_hide, crop_threshold_rect, create_rect, \
                                      direct_max_zproj

In [ ]:
logger = init_logger(r"Palmskin Preprocess")
lif_name_checker = LIFNameChecker()

repo_root = get_repo_root()
print(f"Repository: '{repo_root}'")

In [ ]:
orig_loc = os.getcwd()
orig_stdout = sys.stdout # store original 'sys.stdout'

Load `db_path_plan.toml`

In [ ]:
dbpp_config = load_config("db_path_plan.toml")

db_root      = Path(dbpp_config["root"])
fiji_app_dir = Path(dbpp_config["fiji_app"])
data_nasdl   = dbpp_config["data_nasdl"]

Load `(Preprocess)_palmskin.toml`

In [ ]:
with open("./../../Config/(Preprocess)_palmskin.toml", mode="r") as f_reader:
    config = toml.load(f_reader)

preprocessed_desc = config["data_preprocessed"]["desc"]
preprocessed_reminder = config["data_preprocessed"]["reminder"]

nas_dl_name       = config["data_nas_dl"]["name"]
nas_dl_source_dirs = config["data_nas_dl"]["source_dirs"]

Generate `path_vars`

In [ ]:
UPDATE_MODE = False

# Check `{desc}_Academia_Sinica_i[num]`
data_preprocessed_root = db_root.joinpath(dbpp_config["data_preprocessed"])
target_dir_list = list(data_preprocessed_root.glob(f"*{preprocessed_desc}*"))
assert len(target_dir_list) <= 1, (f"[data_preprocessed.desc] in `(Preprocess)_palmskin.toml` is not unique, "
                                   f"find {len(target_dir_list)} possible directories, {target_dir_list}")
if target_dir_list:
    data_preprocessed_dir = target_dir_list[0]
    UPDATE_MODE = True
else:
    data_preprocessed_dir = data_preprocessed_root.joinpath(f"{{{preprocessed_desc}}}_Academia_Sinica_iTBA")


# Check `{reminder}_PalmSkin_preprocess`
target_dir_list = list(data_preprocessed_dir.glob(f"*PalmSkin_preprocess*"))
assert len(target_dir_list) <= 1, (f"Too many directories are found, only one `PalmSkin_preprocess` is accepted. "
                                   f"Directories: {target_dir_list}")
if target_dir_list:
    palmskin_preprocess_dir = target_dir_list[0]
else:
    palmskin_preprocess_dir = data_preprocessed_dir.joinpath(f"{{{preprocessed_reminder}}}_PalmSkin_preprocess")


create_new_dir(palmskin_preprocess_dir, display_in_CLI=False)
recyclebin_dir = palmskin_preprocess_dir.joinpath("!~delete")
create_new_dir(recyclebin_dir, display_in_CLI=False)

# -----------------------------------------------------------------------------------
raw_lif_dir = db_root.joinpath(dbpp_config["data_nasdl"], nas_dl_name, "palmskin_RGB_RAW")
assert raw_lif_dir.exists(), f"Can't find '{raw_lif_dir}'"

# Initialize PyImageJ

In [ ]:
# Import module
import jpype
# Enable Java imports
import jpype.imports
# Pull in types
from jpype.types import *

import scyjava as sj # scyjava : Supercharged Java access from Python, see https://github.com/scijava/scyjava
import imagej

# Configurations
# NOTE: The ImageJ2 gateway is initialized through a Java Virtual Machine (JVM). If you want to configure the JVM, it must be done before initializing an ImageJ2 gateway.
# sj.config.add_option('-Xmx10g') # adjust memory available to Java
sj.config.endpoints.append('ome:formats-gpl:6.11.1')

In [ ]:
Fiji_Local = str(fiji_app_dir)

# ij = imagej.init(Fiji_Local) # Same as "ij = imagej.init(Fiji_Local, mode='headless')", PyImageJ’s default mode is headless
# ij = imagej.init(Fiji_Local, mode='gui') # GUI mode (會卡在這一行 -> blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#gui-mode
ij = imagej.init(Fiji_Local, mode='interactive') # Interactive mode (可以繼續向下執行 -> non-blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#interactive-mode
ij.ui().showUI() # display the Fiji GUI

print(ij.getApp().getInfo(True)) # ImageJ2 2.9.0/1.54b

In [ ]:
# Bio-Format Reader
loci = jpype.JPackage("loci")
loci.common.DebugTools.setRootLevel("ERROR")
Reader = loci.formats.ImageReader()

# [IMPORTANT] some ImageJ plugins need to be new before use
from ij.plugin.frame import RoiManager
from ij.plugin import ImageCalculator
from ij.plugin import ChannelSplitter
from ij.plugin import ZProjector
from ij.plugin import RGBStackMerge
from ij.plugin import RGBStackConverter

rm = RoiManager()
imageCalculator = ImageCalculator()
channelSplitter = ChannelSplitter()
zProjector = ZProjector()
rgbStackMerge = RGBStackMerge()
rgbStackConverter = RGBStackConverter()

In [ ]:
os.chdir(orig_loc)
sys.stdout = orig_stdout

Test : RGB image name 

In [ ]:
# test_string = r"Series001_fish_1_palmskin_8dpf_A"
test_string = r"Series001_fish_165_A_RG"


# Check format of 'image_name'
detect_old_new_list = re.findall("[RGB]", test_string)
# print(detect_old_new_list)
image_name_list = re.split(" |_|-", test_string)
if len(detect_old_new_list) > 0: failed_cnt, check_dict = check_image_name(image_name_list, "new_rgb", debug_mode=True)
else: failed_cnt, check_dict = check_image_name(image_name_list, "old_rgb", debug_mode=True)


# Consociate image_name
if (check_dict["    format_and_type     "] == "old_rgb") and (failed_cnt == 0): # name check passed, generate new name
    image_name_list.pop(3) # palmskin (old name only)
    image_name_list.pop(3) # [num]dpf (old name only)
    image_name_list.append("RGB")
image_name = "_".join(image_name_list)


# logger.info() --> ERRORs when checking 'image_name'
if failed_cnt == -1 : logger.info(check_dict["  len(image_name_list)  "])
if failed_cnt > 0 : logger.info(check_dict["     failed warning     "])

# log_writer.write() --> ERRORs when checking 'image_name'
if failed_cnt == -1 : print("| {} \n".format(check_dict["  len(image_name_list)  "].strip()))
if failed_cnt > 0: print("| {} \n".format(check_dict["     failed warning     "].strip()))

# single_palmskin_preprocess

In [ ]:
def single_palmskin_preprocess(lif_path:str, total_lif_file:int, out_dir_root:str, logger:logging.Logger, log_writer:TextIO, 
                               Kuwahara_sampleing:int, bf_zproj_type:str, bf_treshold_value:int, relative_mask:str):
    
    
    logger.info(f'LIF_FILE : {lif_path}')
    
    Reader.setId(lif_path)
    seriesCount = Reader.getSeriesCount()
    # logger.info(f'seriesCount : {seriesCount}')
    
    
    for idx in range(seriesCount):
        
        SeriesNum = idx+1
        
        ij.IJ.run("Bio-Formats Importer", f"open='{lif_path}' color_mode=Default rois_import=[ROI manager] view=Hyperstack stack_order=XYCZT series_{SeriesNum}")
        img = ij.WindowManager.getCurrentImage() # get image, <java class 'ij.ImagePlus'>
        # dump_info(img)
        # img.show()
        img.hide()
        
        
        # Get names and image dimensions
        
        # Consociate "palmskin_RGB_RAW" file_name
        file_name = lif_path.split(os.sep)[-1].split(".")[0]
        file_name_list = re.split(" |_|-", file_name)
        assert len(file_name_list) == 4, f"file_name format error, current : '{file_name}', expect like : '20221125_AI005_palmskin_10dpf.lif'"
        file_name = "_".join(file_name_list)
        
        # Get Prop("Image name")
        image_name = img.getProp("Image name")
        image_name = str(image_name)
        
        # Check format of 'image_name'
        detect_old_new_list = re.findall("[RGB]", image_name)
        # print(detect_old_new_list)
        image_name_list = re.split(" |_|-", image_name)
        if len(detect_old_new_list) > 0: failed_cnt, check_dict = check_image_name(image_name_list, "new_rgb")
        else: failed_cnt, check_dict = check_image_name(image_name_list, "old_rgb")
        
        # Consociate image_name
        if (check_dict["    format_and_type     "] == "old_rgb") and (failed_cnt == 0): # name check passed, generate new name
            image_name_list.pop(3) # palmskin (old name only)
            image_name_list.pop(3) # [num]dpf (old name only)
            image_name_list.append("RGB")
        image_name = "_".join(image_name_list)
        
        # Combine 2 names above
        seN = f"{file_name} - {image_name}"
        
        # Get dimension info
        img_dimensions = img.getDimensions()
        z_length = img.getNumericProperty("Image #0|DimensionDescription #6|Length")
        z_slice = img.getNumericProperty("Image #0|DimensionDescription #6|NumberOfElements")
        voxel_z = z_length/(z_slice-1)*(10**6)
        logger.info(f"series {SeriesNum:{len(str(seriesCount))}}/{seriesCount} : '{seN}' , Dimensions : {img_dimensions} ( width, height, channels, slices, frames ), Voxel_Z : {voxel_z:.4f} micron")
        
        # Print ERRORs when checking 'image_name'
        if failed_cnt == -1 : logger.info(check_dict["  len(image_name_list)  "])
        if failed_cnt > 0 : logger.info(check_dict["     failed warning     "])
        
        # Write Log
        log_writer.write(f"|-- processing ...  series {SeriesNum:{len(str(seriesCount))}}/{seriesCount} in {SeriesNum}/{total_lif_file} \n")
        log_writer.write(f"|         {seN} \n")
        log_writer.write(f"|         Dimensions : {img_dimensions} ( width, height, channels, slices, frames ), Voxel_Z : {voxel_z:.4f} micron \n")
        if failed_cnt == -1 : log_writer.write("| {} \n".format(check_dict["  len(image_name_list)  "].strip()))
        if failed_cnt > 0: log_writer.write("| {} \n".format(check_dict["     failed warning     "].strip()))
        
        
        # Start image preprocessing
        
        
        # preprocessing var
        # Kuwahara_sampleing = 15
        # bf_zproj_type = "median" # 'method' is "avg", "min", "max", "sum", "sd" or "median".
        # bf_treshold_value = 10
        # relative_mask = "outer" # "inner" or "outer"
        tif_save_cnt = 0
        tif_save_SNdigits = "02"
        
        
        # Create sub folder, MetaImage folder
        if "delete" in seN: subfolder = os.path.join(out_dir_root, "!~delete", seN)
        else: subfolder = os.path.join(out_dir_root, seN)
        assert not Path(subfolder).exists(), f"target directory exists: '{subfolder}'"
        dir_metaimg = os.path.join(subfolder, "MetaImage")
        create_new_dir(subfolder, display_in_CLI=False)
        create_new_dir(dir_metaimg, display_in_CLI=False)
        
        
        ij.IJ.run(img, "Set Scale...", "distance=2.2 known=1 unit=micron")
        # img.show()
        ch_list = channelSplitter.split(img)
        # -----------------------------------------------------------------------------------
        RGB_direct_max_zproj = direct_max_zproj(ch_list, zProjector, rgbStackMerge, rgbStackConverter)
        tif_save_cnt = save_tif_with_SN(RGB_direct_max_zproj, "RGB_direct_max_zproj", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        
        # -----------------------------------------------------------------------------------
        ch_B = ch_list[0]
        # processed
        B_processed = median_R1_and_mean3D_R2(ch_B, zProjector, ij)
        tif_save_cnt = save_tif_with_SN(B_processed, "B_processed", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        # processed + Kuwahara Filter
        B_processed_Kuwahara = B_processed.duplicate()
        ij.IJ.run(B_processed_Kuwahara, "Kuwahara Filter", f"sampling={Kuwahara_sampleing}")
        tif_save_cnt = save_tif_with_SN(B_processed_Kuwahara, f"B_processed_Kuwahara{Kuwahara_sampleing}", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        # processed + HE
        B_processed_HE = B_processed.duplicate()
        ij.IJ.run(B_processed_HE, "Enhance Contrast...", "saturated=0.35 equalize")
        tif_save_cnt = save_tif_with_SN(B_processed_HE, "B_processed_HE", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        # processed + Kuwahara Filter + HE
        B_processed_Kuwahara_HE = B_processed_Kuwahara.duplicate()
        ij.IJ.run(B_processed_Kuwahara_HE, "Enhance Contrast...", "saturated=0.35 equalize")
        tif_save_cnt = save_tif_with_SN(B_processed_Kuwahara_HE, f"B_processed_Kuwahara{Kuwahara_sampleing}_HE", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        # Average 2 Non-HE images
        B_processed_fusion = imageCalculator.run(B_processed, B_processed_Kuwahara, "Average create")
        tif_save_cnt = save_tif_with_SN(B_processed_fusion, "B_processed_fusion", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        # Average 2 HE images
        B_processed_HE_fusion = imageCalculator.run(B_processed_HE, B_processed_Kuwahara_HE, "Average create")
        tif_save_cnt = save_tif_with_SN(B_processed_HE_fusion, "B_processed_HE_fusion", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        
        
        # -----------------------------------------------------------------------------------
        ch_G = ch_list[1]
        # processed
        G_processed = median_R1_and_mean3D_R2(ch_G, zProjector, ij)
        tif_save_cnt = save_tif_with_SN(G_processed, "G_processed", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        # processed + Kuwahara Filter
        G_processed_Kuwahara = G_processed.duplicate()
        ij.IJ.run(G_processed_Kuwahara, "Kuwahara Filter", f"sampling={Kuwahara_sampleing}")
        tif_save_cnt = save_tif_with_SN(G_processed_Kuwahara, f"G_processed_Kuwahara{Kuwahara_sampleing}", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        # processed + HE
        G_processed_HE = G_processed.duplicate()
        ij.IJ.run(G_processed_HE, "Enhance Contrast...", "saturated=0.35 equalize")
        tif_save_cnt = save_tif_with_SN(G_processed_HE, "G_processed_HE", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        # processed + Kuwahara Filter + HE
        G_processed_Kuwahara_HE = G_processed_Kuwahara.duplicate()
        ij.IJ.run(G_processed_Kuwahara_HE, "Enhance Contrast...", "saturated=0.35 equalize")
        tif_save_cnt = save_tif_with_SN(G_processed_Kuwahara_HE, f"G_processed_Kuwahara{Kuwahara_sampleing}_HE", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        # Average 2 Non-HE images
        G_processed_fusion = imageCalculator.run(G_processed, G_processed_Kuwahara, "Average create")
        tif_save_cnt = save_tif_with_SN(G_processed_fusion, "G_processed_fusion", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        # Average 2 HE images
        G_processed_HE_fusion = imageCalculator.run(G_processed_HE, G_processed_Kuwahara_HE, "Average create")
        tif_save_cnt = save_tif_with_SN(G_processed_HE_fusion, "G_processed_HE_fusion", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        
        
        # -----------------------------------------------------------------------------------
        ch_R = ch_list[2]
        # processed
        R_processed = median_R1_and_mean3D_R2(ch_R, zProjector, ij)
        tif_save_cnt = save_tif_with_SN(R_processed, "R_processed", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        # processed + Kuwahara Filter
        R_processed_Kuwahara = R_processed.duplicate()
        ij.IJ.run(R_processed_Kuwahara, "Kuwahara Filter", f"sampling={Kuwahara_sampleing}")
        tif_save_cnt = save_tif_with_SN(R_processed_Kuwahara, f"R_processed_Kuwahara{Kuwahara_sampleing}", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        # processed + HE
        R_processed_HE = R_processed.duplicate()
        ij.IJ.run(R_processed_HE, "Enhance Contrast...", "saturated=0.35 equalize")
        tif_save_cnt = save_tif_with_SN(R_processed_HE, "R_processed_HE", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        # processed + Kuwahara Filter + HE
        R_processed_Kuwahara_HE = R_processed_Kuwahara.duplicate()
        ij.IJ.run(R_processed_Kuwahara_HE, "Enhance Contrast...", "saturated=0.35 equalize")
        tif_save_cnt = save_tif_with_SN(R_processed_Kuwahara_HE, f"R_processed_Kuwahara{Kuwahara_sampleing}_HE", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        # Average 2 Non-HE images
        R_processed_fusion = imageCalculator.run(R_processed, R_processed_Kuwahara, "Average create")
        tif_save_cnt = save_tif_with_SN(R_processed_fusion, "R_processed_fusion", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        # Average 2 HE images
        R_processed_HE_fusion = imageCalculator.run(R_processed_HE, R_processed_Kuwahara_HE, "Average create")
        tif_save_cnt = save_tif_with_SN(R_processed_HE_fusion, "R_processed_HE_fusion", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        
        
        # -----------------------------------------------------------------------------------
        # Merge, Enhance Contrast, without Kuwahara Filter
        # group_show(R_processed, G_processed, B_processed) # needs to show image before merge
        # ij.IJ.run("Merge Channels...", "c1=[05_R_processed.tif] c2=[03_G_processed.tif] c3=[01_B_processed.tif] create keep")
        # RGB_processed = ij.WindowManager.getCurrentImage()
        RGB_processed = rgbStackMerge.mergeChannels([R_processed, G_processed, B_processed], True)
        # group_hide(R_processed, G_processed, B_processed)
        # ij.IJ.run(RGB_processed, "RGB Color", "") # will create new window
        # RGB_processed_HE = ij.WindowManager.getCurrentImage()
        rgbStackConverter.convertToRGB(RGB_processed)
        tif_save_cnt = save_tif_with_SN(RGB_processed, "RGB_processed", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        RGB_processed_HE = RGB_processed.duplicate()
        ij.IJ.run(RGB_processed_HE, "Enhance Contrast...", "saturated=0.35 equalize")
        tif_save_cnt = save_tif_with_SN(RGB_processed_HE, "RGB_processed_HE", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        # group_hide(RGB_processed, RGB_processed_HE)
        
        
        # -----------------------------------------------------------------------------------
        # Merge, Enhance Contrast, with Kuwahara Filter
        # group_show(R_processed_Kuwahara, G_processed_Kuwahara, B_processed_Kuwahara) # needs to show image before merge
        # ij.IJ.run("Merge Channels...", f"c1=[06_R_processed_Kuwahara{Kuwahara_sampleing}.tif] c2=[04_G_processed_Kuwahara{Kuwahara_sampleing}.tif] c3=[02_B_processed_Kuwahara{Kuwahara_sampleing}.tif] create keep")
        # RGB_processed_Kuwahara = ij.WindowManager.getCurrentImage()
        RGB_processed_Kuwahara = rgbStackMerge.mergeChannels([R_processed_Kuwahara, G_processed_Kuwahara, B_processed_Kuwahara], True)
        # group_hide(R_processed_Kuwahara, G_processed_Kuwahara, B_processed_Kuwahara)
        # ij.IJ.run(RGB_processed_Kuwahara, "RGB Color", "") # will create new window
        # RGB_processed_Kuwahara_HE = ij.WindowManager.getCurrentImage()
        rgbStackConverter.convertToRGB(RGB_processed_Kuwahara)
        tif_save_cnt = save_tif_with_SN(RGB_processed_Kuwahara, f"RGB_processed_Kuwahara{Kuwahara_sampleing}", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        RGB_processed_Kuwahara_HE = RGB_processed_Kuwahara.duplicate()
        ij.IJ.run(RGB_processed_Kuwahara_HE, "Enhance Contrast...", "saturated=0.35 equalize")
        tif_save_cnt = save_tif_with_SN(RGB_processed_Kuwahara_HE, f"RGB_processed_Kuwahara{Kuwahara_sampleing}_HE", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        # group_hide(RGB_processed_Kuwahara, RGB_processed_Kuwahara_HE)
        
        
        # -----------------------------------------------------------------------------------
        # Average 2 Non-HE images and save in RGB, weighted-grayscale
        RGB_processed_fusion = imageCalculator.run(RGB_processed, RGB_processed_Kuwahara, "Average create")
        tif_save_cnt = save_tif_with_SN(RGB_processed_fusion, "RGB_processed_fusion", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        RGB_processed_fusion2Gray = RGB_processed_fusion.duplicate()
        ij.IJ.run("Conversions...", "scale weighted")
        ij.IJ.run(RGB_processed_fusion2Gray, "8-bit", "")
        tif_save_cnt = save_tif_with_SN(RGB_processed_fusion2Gray, "RGB_processed_fusion2Gray", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        
        # Average 2 HE images and save in RGB, weighted-grayscale
        RGB_processed_HE_fusion = imageCalculator.run(RGB_processed_HE, RGB_processed_Kuwahara_HE, "Average create")
        tif_save_cnt = save_tif_with_SN(RGB_processed_HE_fusion, "RGB_processed_HE_fusion", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        RGB_processed_HE_fusion2Gray = RGB_processed_HE_fusion.duplicate()
        ij.IJ.run("Conversions...", "scale weighted")
        ij.IJ.run(RGB_processed_HE_fusion2Gray, "8-bit", "")
        tif_save_cnt = save_tif_with_SN(RGB_processed_HE_fusion2Gray, "RGB_processed_HE_fusion2Gray", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        
        
        # -----------------------------------------------------------------------------------
        # Generate Mask using Bright Field
        ch_BF = ch_list[3]
        BF_Zproj = zProjector.run(ch_BF, bf_zproj_type) # 'method' is "avg", "min", "max", "sum", "sd" or "median".
        tif_save_cnt = save_tif_with_SN(BF_Zproj, f"BF_Zproj_{bf_zproj_type}", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        ij.IJ.run(BF_Zproj, "Enhance Contrast...", "saturated=0.35 equalize")
        tif_save_cnt = save_tif_with_SN(BF_Zproj, f"BF_Zproj_{bf_zproj_type}_HE", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        ij.IJ.setRawThreshold(BF_Zproj, 0, bf_treshold_value)
        ij.IJ.run(BF_Zproj, "Convert to Mask", "")
        tif_save_cnt = save_tif_with_SN(BF_Zproj, f"Threshold_0_{bf_treshold_value}", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        threshold_img = imread(f"{dir_metaimg}/{tif_save_cnt-1:{tif_save_SNdigits}}_Threshold_0_{bf_treshold_value}.tif") # use scikit-image
        outer_rect = create_rect(threshold_img, "outer") # use scikit-image
        inner_rect = create_rect(threshold_img, "inner") # use scikit-image
        outer_rect = ij.py.to_imageplus(outer_rect)
        inner_rect = ij.py.to_imageplus(inner_rect)
        tif_save_cnt = save_tif_with_SN(outer_rect, f"outer_rect", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        tif_save_cnt = save_tif_with_SN(inner_rect, f"inner_rect", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        if relative_mask == "outer": auto_rect = outer_rect
        else: auto_rect = inner_rect
        auto_rect = ij.py.to_imageplus(auto_rect)
        ij.IJ.run("Set Measurements...", "area mean min feret's display redirect=None decimal=2")
        ij.IJ.run(auto_rect, "Analyze Particles...", "size=0-Infinity show=Nothing display include add")
        
        
        # Check ROI
        rm.runCommand("Show All with labels")
        rm_size = int(rm.getCount())
        if rm_size == 1:
            rm.save(os.path.normpath(f"{dir_metaimg}/RoiSet_AutoRect.roi"))
        else:
            logger.info(f'      ROI in RoiManager: {rm_size}')
            logger.info(f'      #### ERROR : Number of ROI not = 1')
            # Write Log
            log_writer.write(f"|         number of ROI = {rm_size} \n")
            log_writer.write("| #### ERROR : Number of ROI not = 1 \n") # ERROR:
        
        
        # # Fit ROI with Rect 
        # auto_rect = mask_img.duplicate()
        # ij.IJ.setForegroundColor(255, 255, 255)
        # ij.IJ.setBackgroundColor(0, 0, 0)
        # auto_rect.show()
        # auto_rect.setRoi(rm.getRoi(0))
        # ij.IJ.run(auto_rect, "Fit Rectangle", "")
        # ij.IJ.run(auto_rect, "Fill", "slice")
        # auto_rect.hide()
        
        
        # Average with `auto_rect`
        RGB_processed_fusion_AvgAutoRect = imageCalculator.run(RGB_processed_fusion, auto_rect, "Average create")
        tif_save_cnt = save_tif_with_SN(RGB_processed_fusion_AvgAutoRect, f"RGB_processed_fusion--AutoRect", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        RGB_processed_HE_fusion_AvgAutoRect = imageCalculator.run(RGB_processed_HE_fusion, auto_rect, "Average create")
        tif_save_cnt = save_tif_with_SN(RGB_processed_HE_fusion_AvgAutoRect, f"RGB_processed_HE_fusion--AutoRect", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        
        
        # crop_threshold_roi, save_as_tiff
        autocropped_RGB_processed_fusion = crop_threshold_rect(RGB_processed_fusion, rm.getRoi(0), ij)
        tif_save_cnt = save_tif_with_SN(autocropped_RGB_processed_fusion, f"autocropped_RGB_processed_fusion", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        autocropped_RGB_processed_HE_fusion = crop_threshold_rect(RGB_processed_HE_fusion, rm.getRoi(0), ij)
        tif_save_cnt = save_tif_with_SN(autocropped_RGB_processed_HE_fusion, f"autocropped_RGB_processed_HE_fusion", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        
        
        # Show image
        # composite_HE_mix_AvgMaskRect.show()
        # composite_HE_mix_AvgMaskRect.hide()
        
        rm.runCommand("Deselect")
        rm.runCommand("Delete")
        ij.IJ.run("Clear Results", "")
        ij.IJ.run("Close All", "")
        log_writer.write(f"| \n") # make Log file looks better.

# Start Process

In [ ]:
if int(rm.getCount()) > 0:
    rm.runCommand("Deselect")
    rm.runCommand("Delete")
    ij.IJ.run("Clear Results", "")
ij.IJ.run("Close All", "")


logger.info("MODE: UPDATE") if UPDATE_MODE else logger.info("MODE: CREATE")

# Scan Leica LIF file
lif_path_list = []
if nas_dl_source_dirs:
    for source_dir in nas_dl_source_dirs:
        temp_list = list(raw_lif_dir.joinpath(source_dir).glob("**/*.lif"))
        lif_path_list.extend(temp_list)
else:
    lif_path_list = list(raw_lif_dir.glob("**/*.lif"))
lif_path_list.sort(key=lambda x: str(x).split(os.sep)[-1])
# lif_path_list = lif_path_list[:1]
for lif_path in lif_path_list: 
    logger.info(f'lif_path_list {type(lif_path)}: {lif_path}')
logger.info(f"[ found {len(lif_path_list)} lif files ]")


# Create Log writer
time_stamp = datetime.now().strftime('%Y%m%d_%H_%M_%S')
log_path = os.path.join(palmskin_preprocess_dir, f"{{Logs}}_{{ij_palmskin_preprocess}}_{time_stamp}.log")
log_writer = open(log_path, mode="w")
logger.info(f'log_writer {type(log_writer)}: {log_writer} \n')


config_save_path = palmskin_preprocess_dir.joinpath("palmskin_preprocess_config.toml")
if UPDATE_MODE:
    assert config_save_path.exists(), f"UpdateError, Can't find existing config: '{config_save_path}'" # find existing config
    with open(config_save_path, mode="r") as f_reader:
        preprocess_kwargs = toml.load(f_reader)["param"]
else:
    preprocess_kwargs = config["param"]
    with open(config_save_path, mode="w") as f_writer:
        tomlkit.dump(config, f_writer)


for i, lif_path in enumerate(lif_path_list):
    
    lif_path = str(lif_path)
    logger.info(f"Processing ... {i+1}/{len(lif_path_list)}")
    
    # Write Log
    log_writer.write("\n\n\n")
    log_writer.write(f"|-----------------------------------------  Processing ... {i+1}/{len(lif_path_list)}  ----------------------------------------- \n")
    log_writer.write(f"| \n")
    log_writer.write(f"|         LIF_FILE : {lif_path.split(os.sep)[-1]} \n")
    log_writer.write(f"| \n")
    
    # process current LIF_FILE
    single_palmskin_preprocess(lif_path, len(lif_path_list), palmskin_preprocess_dir, logger, log_writer, **preprocess_kwargs)
    logger.info("\n") # make CLI output looks better.


logger.info(" -- finished --")
log_writer.write("\n\n\n-----------------------------------------  finished  ----------------------------------------- \n")
log_writer.close()
# rm.close()
# ij.WindowManager.closeAllWindows()

In [ ]:
log_writer.close()